# Linear Regression with PyTorch: Datasets, Models, and Losses 
Author: Pierre Nugues

## Dataset

We extract the counts of letters per chapter and the counts of _A_ from the *Salammbô* novel by Flaubert. There are 15 chapters in total.

In [1]:
import torch
from torch import nn
from torch import optim
import random
import plotly.graph_objects as go

In [2]:
random.seed(4321)
torch.manual_seed(4321)

The $X$ matrix

In [3]:
X = torch.Tensor(
    [[36961],
     [43621],
     [15694],
     [36231],
     [29945],
     [40588],
     [75255],
     [37709],
     [30899],
     [25486],
     [37497],
     [40398],
     [74105],
     [76725],
     [18317]])

In the mean squared error loss function (MSE), $y$ must have the same shape as the input. See here: `https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html`. We reshape it so that it becomes a column vector.

In [4]:
y = torch.Tensor(
    [2503, 2992, 1042, 2487, 2014, 2805, 5062, 2643, 2126, 1784, 2641, 2766,
     5047, 5312, 1215]).reshape((-1, 1))
y

tensor([[2503.],
        [2992.],
        [1042.],
        [2487.],
        [2014.],
        [2805.],
        [5062.],
        [2643.],
        [2126.],
        [1784.],
        [2641.],
        [2766.],
        [5047.],
        [5312.],
        [1215.]])

## Visualizing the Dataset

In [5]:
X.squeeze(dim=1)

tensor([36961., 43621., 15694., 36231., 29945., 40588., 75255., 37709., 30899.,
        25486., 37497., 40398., 74105., 76725., 18317.])

In [6]:
y

tensor([[2503.],
        [2992.],
        [1042.],
        [2487.],
        [2014.],
        [2805.],
        [5062.],
        [2643.],
        [2126.],
        [1784.],
        [2641.],
        [2766.],
        [5047.],
        [5312.],
        [1215.]])

In [7]:
fig = go.Figure(data=[
    go.Scatter(
        x=X.flatten(),
        y=y.flatten(),
        mode='markers',
        marker=dict(color='blue', symbol='x'),
        name='Data'
    )
])

fig.update_layout(
    title="Salammbô",
    xaxis_title="Letter count",
    yaxis_title="A count",
)

fig.show()

## Linear Regression with PyTorch

We create the architecture. The model has an intercept (a bias) by default.

In [8]:
model = nn.Sequential(nn.Linear(1, 1))
model

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

In [9]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[-0.7489]], requires_grad=True),
 Parameter containing:
 tensor([0.0753], requires_grad=True))

We use the mean squared error and nadam, a variant of stochastic gradient descent, to find the parameters

In [10]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.7489]], requires_grad=True),
 Parameter containing:
 tensor([0.0753], requires_grad=True)]

In [11]:
loss_fn = nn.MSELoss()
optimizer = optim.NAdam(model.parameters(), lr=0.01)

## Batch descent

We fit the two parameters with the whole dataset

In [12]:
loss_fn

MSELoss()

In [13]:
optimizer

NAdam (
Parameter Group 0
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    lr: 0.01
    maximize: False
    momentum_decay: 0.004
    weight_decay: 0
)

In [14]:
model

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

In [15]:
loss_history = []
model_x = []
model_y = []
model.train()
for epoch in range(250):
    sse = 0
    y_pred = model(X)                       # We compute Xw = y_hat
    loss = loss_fn(y_pred, y)               # (y_hat - y)^2
    sse = loss.item()
    model_x += [model[0].bias.item()]
    model_y += [model[0].weight.item()]
    loss_history += [sse]
    optimizer.zero_grad()
    loss.backward()                         # we compute the gradients ∇ (h_hat - y)^2
    optimizer.step()                        # we update the weights -⍺ ∇ (h_hat - y)^2

In [16]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[0.0676]], requires_grad=True),
 Parameter containing:
 tensor([0.8920], requires_grad=True))

In [17]:
model.eval()

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

Predict the values with the model: $model(X_{\text{test}})$

In [18]:
X_test = torch.FloatTensor([[3000],
                            [5000]])

In [19]:
with torch.no_grad():
    print(model(X_test))

tensor([[203.5815],
        [338.7079]])


Predict one input `X_test[0]` with the parameters: $w_1 x + w_0$

In [20]:
with torch.no_grad():
    print(model[0].weight @ X_test[0] +
          model[0].bias)

tensor([203.5815])


Predict a dataset: $XW^\intercal + w_0$

In [21]:
with torch.no_grad():
    print(X_test @ model[0].weight.T +
          model[0].bias)

tensor([[203.5815],
        [338.7079]])


### The Model History

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=model_x,
    y=model_y,
    mode='markers',
    marker=dict(
        color='red',
        symbol='x',

    ),
    name='weight vector'
))
fig.update_layout(
    title='Salammbô',
    xaxis_title='bias',
    yaxis_title='weight',
    showlegend=True,
)

### The Loss History

In [23]:
loss_history[-1]

3708.039794921875

Last loss after last update

In [24]:
with torch.no_grad():
    last_loss = (model(X) - y).T @ (model(X) - y)/X.size()[0]
last_loss.item()

3551.81103515625

### Visualising the Loss
We visualise the loss during the training process

In [25]:
epochs = range(1, len(loss_history) + 1)

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(epochs),
    y=loss_history,
    mode='lines+markers',
    marker=dict(
        color='blue',
        symbol='circle'
    ),
    line=dict(color='blue'),
    name='Training loss'
))

fig.update_layout(
    title='Training loss',
    xaxis_title='epochs',
    yaxis_title='loss',
    showlegend=True
)

### Visualization of the model evolution on the loss surface

In [27]:
def mean_squared_errors(X: torch.tensor, y: torch.tensor, w: torch.tensor) -> torch.tensor:
    """
    Sum of the squared errors:
    Prediction: Xw = ŷ
    Error (loss): (y - ŷ)^2
    :param X: The input matrix: The predictors
    :param y: The output vector: The response
    :param w: The weight vector: The model (3D as it is a grid)
    :return: The error
    """

    original_size = torch.tensor(w.size()[1:])  # (100, 100)
    num_points = torch.prod(original_size)  # 10000
    w = w.reshape(w.size(0), num_points)  # (2, 10000)

    y_pred = X @ w  # We predict y for all the weights in our simulation

    # We align the size of y with that of ŷ
    y_repeated = y.repeat(1, y_pred.size(1))  # (15, num_points)
    error = y_repeated - y_pred

    mse = torch.log10((error ** 2).mean(dim=0))  # (y - ŷ)^2/N
    mse = mse.reshape(original_size.tolist())
    return mse

In [28]:
w0_range = torch.linspace(-1000, 1000, 100)
w1_range = torch.linspace(0.0, 0.15, 100)
X_fig, Y_fig = torch.meshgrid(w0_range, w1_range, indexing='xy')
w_grid = torch.stack([X_fig, Y_fig])  # (2, 100, 100)

In [29]:
# We add 1.0 to take the bias into account (homogeneous coordinates)
X_bias = torch.cat((torch.ones((len(X), 1)), X), dim=1)  # (15, 2)

Z_fig = mean_squared_errors(X_bias, y, w_grid)

In [30]:
fig = go.Figure(
    data=[go.Surface(z=Z_fig, x=X_fig, y=Y_fig, colorscale='viridis')])
fig.update_layout(
    scene=dict(
        xaxis_title='bias',
        yaxis_title='weight',
        zaxis_title='Log10(Loss)'
    )
)

In [31]:
fig

In [32]:
fig.add_trace(go.Scatter3d(
    x=model_x,
    y=model_y,
    z=torch.log10(torch.tensor(loss_history)),
    mode='markers',
    marker=dict(
        color='red',
        symbol='circle',
        size=1
    ),
    name='Points loss'
))

fig.update_layout(
    scene=dict(
        xaxis_title='bias',
        yaxis_title='weight',
        zaxis_title='Log10(Loss)'
    )
)

fig.show()

In [33]:
model_x[-1], model_y[-1], loss_history[-1]

(0.8919244408607483, 0.06752170622348785, 3708.039794921875)

## Stochastic descent
Each observation will lead to an update

We reset the model

In [34]:
model[0].reset_parameters()

In [35]:
loss_history = []
model_x = []
model_y = []
model.train()
for epoch in range(20):
    sse = 0
    for xi, yi in zip(X, y):
        yi_pred = model(xi)
        loss = loss_fn(yi_pred, yi)
        sse += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_history += [sse/X.size()[0]]
    model_x += [model[0].bias.item()]
    model_y += [model[0].weight.item()]

In [36]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[0.0685]], requires_grad=True),
 Parameter containing:
 tensor([-1.1780], requires_grad=True))

### Visualising the Loss
We visualise the loss during the training process

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(epochs),
    y=loss_history,
    mode='lines+markers',
    marker=dict(
        color='blue',
        symbol='circle'
    ),
    line=dict(color='blue'),
    name='Training loss'
))

fig.update_layout(
    title='Training loss',
    showlegend=True
)

In [38]:
fig = go.Figure(
    data=[go.Surface(z=Z_fig, x=X_fig, y=Y_fig, colorscale='viridis')])

In [39]:
fig.add_trace(go.Scatter3d(
    x=model_x,
    y=model_y,
    z=torch.log10(torch.tensor(loss_history)),
    mode='markers',
    marker=dict(
        color='red',
        symbol='circle',
        size=5
    ),
    name='Points loss'
))

fig.update_layout(
    scene=dict(
        xaxis_title='bias',
        yaxis_title='weight',
        zaxis_title='Log10(Loss)'
    )
)

fig.show()

## Mini-batch gradient descent

In [40]:
BATCH_SIZE = 4

In [41]:
model[0].reset_parameters()

In [42]:
loss_history = []
model_x = []
model_y = []
model.train()
for epoch in range(40):
    sse = 0
    for i in range(X.size()[0]//BATCH_SIZE):
        Xi = X[i:i+BATCH_SIZE]
        yi = y[i:i+BATCH_SIZE]
        yi_pred = model(Xi)
        loss = loss_fn(yi_pred, yi)
        sse += loss.item() * Xi.size()[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_history += [sse/X.size()[0]]
    model_x += [model[0].bias.item()]
    model_y += [model[0].weight.item()]

In [43]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[0.0696]], requires_grad=True),
 Parameter containing:
 tensor([0.2620], requires_grad=True))

### Loss History

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(epochs),
    y=loss_history,
    mode='lines+markers',
    marker=dict(
        color='blue',
        symbol='circle'
    ),
    line=dict(color='blue'),
    name='Training loss'
))

fig.update_layout(
    title='Training loss',
    showlegend=True
)

In [45]:
fig = go.Figure(
    data=[go.Surface(z=Z_fig, x=X_fig, y=Y_fig, colorscale='viridis')])

In [46]:
fig.add_trace(go.Scatter3d(
    x=model_x,
    y=model_y,
    z=torch.log10(torch.tensor(loss_history)),
    mode='markers',
    marker=dict(
        color='red',
        symbol='circle',
        size=3
    ),
    name='Points loss'
))

fig.update_layout(
    scene=dict(
        xaxis_title='bias',
        yaxis_title='weight',
        zaxis_title='Log10(Loss)'
    )
)

fig.show()

## With a Dataloader

In [47]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [48]:
model[0].reset_parameters()

In [49]:
loss_history = []
model_x = []
model_y = []
model.train()
for epoch in range(100):
    sse = 0
    for X_batch, y_batch in dataloader:
        y_batch_pred = model(X_batch)
        loss = loss_fn(y_batch_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sse += loss.item() * X_batch.size()[0]
    loss_history += [sse/X.size()[0]]
    model_x += [model[0].bias.item()]
    model_y += [model[0].weight.item()]

In [50]:
model[0].weight, model[0].bias

(Parameter containing:
 tensor([[0.0684]], requires_grad=True),
 Parameter containing:
 tensor([1.3601], requires_grad=True))

In [51]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(epochs),
    y=loss_history,
    mode='lines+markers',
    marker=dict(
        color='blue',
        symbol='circle'
    ),
    line=dict(color='blue'),
    name='Training loss'
))

fig.update_layout(
    title='Training loss',
    showlegend=True
)

In [52]:
fig = go.Figure(
    data=[go.Surface(z=Z_fig, x=X_fig, y=Y_fig, colorscale='viridis')])

In [54]:
fig.add_trace(go.Scatter3d(
    x=model_x,
    y=model_y,
    z=torch.log10(torch.tensor(loss_history)),
    mode='markers',
    marker=dict(
        color='red',
        symbol='circle',
        size=5
    ),
    name='Points loss'
))

fig.update_layout(
    scene=dict(
        xaxis_title='bias',
        yaxis_title='weight',
        zaxis_title='Log10(Loss)'
    )
)

fig.show()

## Visualizing the Final Model

In [55]:
X, y

(tensor([[36961.],
         [43621.],
         [15694.],
         [36231.],
         [29945.],
         [40588.],
         [75255.],
         [37709.],
         [30899.],
         [25486.],
         [37497.],
         [40398.],
         [74105.],
         [76725.],
         [18317.]]),
 tensor([[2503.],
         [2992.],
         [1042.],
         [2487.],
         [2014.],
         [2805.],
         [5062.],
         [2643.],
         [2126.],
         [1784.],
         [2641.],
         [2766.],
         [5047.],
         [5312.],
         [1215.]]))

In [56]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=y.flatten(),
    mode='markers',
    marker=dict(
        color='blue',
        symbol='cross'
    ),
    line=dict(color='blue'),
    name='Training loss'
))
fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=model(X).flatten().detach(),
    mode='markers',
    marker=dict(
        color='red',
        symbol='arrow'
    ),
    line=dict(color='blue'),
    name='Training loss'
))

fig.update_layout(
    title='Salammbô',
    xaxis_title='Letter count',
    yaxis_title='$A$ count',
    showlegend=True,
)

## Gradients
PyTorch computes automatically the gradients. See https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html. Here we will compute them manually and check the PyTorch's values

We fit the model again

In [57]:
model[0].reset_parameters()

In [58]:
loss_history = []
model.train()
for epoch in range(250):
    sse = 0
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    sse = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_history += [sse]

We apply it to the dataset and we compute the loss

In [59]:
model.eval()
y_pred = model(X)
y_pred

tensor([[2531.0288],
        [2987.0583],
        [1074.8154],
        [2481.0435],
        [2050.6228],
        [2779.3801],
        [5153.1309],
        [2582.2466],
        [2115.9460],
        [1745.3022],
        [2567.7302],
        [2766.3701],
        [5074.3872],
        [5253.7861],
        [1254.4199]], grad_fn=<AddmmBackward0>)

The loss according to PyTorch

In [60]:
loss = loss_fn(y_pred, y)
loss

tensor(1905.0551, grad_fn=<MseLossBackward0>)

we check the loss manually

In [61]:
1/X.size()[0] * (y_pred - y).T @ (y_pred - y)

tensor([[1905.0551]], grad_fn=<MmBackward0>)

We compute the gradients manually:
$$
\begin{array}{lclcl}
\displaystyle{\frac{\partial{Loss}}{\partial{m}}} &=& \displaystyle{\sum_{i = 1}^{q}{\frac{\partial{}}{\partial{m}}(y_i - (mx_i + b))^2}} &=& \displaystyle{-2\sum_{i = 1}^{q}{x_i(y_i - (mx_i + b))}} \\
\displaystyle{\frac{\partial{Loss}}{\partial{b}}} &=& \displaystyle{\sum_{i = 1}^{q}{\frac{\partial{}}{\partial{b}}(y_i - (mx_i + b))^2}} &=& \displaystyle{-2 \sum_{i = 1}^{q}{(y_i - (mx_i + b))}}
\end{array}
$$
$m$ is the weight and $b$ is the bias. By default, PyTorch computes the gradient mean of a batch

The gradient according to our equations

In [62]:
-2/X.size()[0] * (y - y_pred).T @ X, -2/X.size()[0] * torch.sum(y - y_pred)

(tensor([[-94.5000]], grad_fn=<MmBackward0>),
 tensor(-2.8976, grad_fn=<MulBackward0>))

### PyTorch's gradients
The gradients are accumulated, we need to clear them before we apply `backward()` to compute them

In [63]:
optimizer.zero_grad()
loss.backward()

The gradient according to PyTorch

In [64]:
model[0].weight.grad, model[0].bias.grad

(tensor([[-94.5000]]), tensor([-2.8976]))